In [ ]:
import xlwings as xw

wb = xw.Book(r"Demo.xlsx")
wb

In [ ]:
watch_list = wb.sheets["觀察清單"]
watch_list

In [ ]:
import requests
def fugle_stock_crawler(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/quote", params=payload)
    json_data = res.json()

    res1 = requests.get("https://api.fugle.tw/realtime/v0/intraday/meta", params=payload)
    json_data1 = res1.json()

    return {
        "open": json_data["data"]["quote"]["priceOpen"]["price"],
        "high": json_data["data"]["quote"]["priceHigh"]["price"],
        "low": json_data["data"]["quote"]["priceLow"]["price"],
        "close": json_data["data"]["quote"]["trade"]["price"],
        "lastClose": json_data1["data"]["meta"]["priceReference"],
        "name": json_data1["data"]["meta"]["nameZhTw"]
    }

In [ ]:
api_token = watch_list.range("L1").value
stock_id = "2884"

print(api_token)
fugle_stock_crawler(stock_id, api_token)

In [ ]:
api_token = watch_list.range("L1").value
stock_id = "2884"

data = fugle_stock_crawler(stock_id, api_token)

[
    data["name"],
    data["open"],
    data["high"],
    data["low"],
    data["close"],
    data["lastClose"]
]

In [ ]:
api_token = watch_list.range("L1").value
stock_id = "2884"

data = fugle_stock_crawler(stock_id, api_token)

watch_list.range("B2").value = [data["name"],data["open"],data["high"],data["low"],data["close"],data["lastClose"]]

In [ ]:
import requests
import numpy as np

def get_chart_data(stock_id, api_token):
    payload = {
        "symbolId": stock_id,
        "apiToken": api_token
    }

    res = requests.get("https://api.fugle.tw/realtime/v0/intraday/chart", params=payload)

    result = res.json()

    price_open = []
    price_high = []
    price_low = []
    price_close = []
    unit = []
    volume = []

    time_idx = result["data"]["chart"].keys()

    for time in time_idx:
        data = result["data"]["chart"]
        price_open.append(data[time]["open"])
        price_high.append(data[time]["high"])
        price_low.append(data[time]["low"])
        price_close.append(data[time]["close"])
        unit.append(data[time]["unit"])
        volume.append(data[time]["volume"])

    real_time_data = [list(time_idx), price_open,price_high,price_low,price_close,unit,volume]
    stock_data_ary = np.array(real_time_data)
    
    return stock_data_ary.T

In [ ]:
data = get_chart_data(2884, "demo")
print(data)
print(data.shape)

In [ ]:
# 將 TW2884 存入 target_sheet
target_sheet = wb.sheets["2884走勢資料"]
# 將二維串列的資料指定給 A2 為左上角的範圍
target_sheet.range("A2").value = get_chart_data(2884, "demo")

In [ ]:
dashboard = wb.sheets["即時看板"]
# 將即時看板内的圖表物件都讀取出來
dashboard.charts

In [ ]:
# 選擇即時看板内名爲 chart1 的圖表，存入 chart 變數
chart = dashboard.charts["chart1"]
chart

In [ ]:
dashboard = wb.sheets["即時看板"]
target_sheet = wb.sheets["2884走勢資料"]

chart = dashboard.charts["chart1"]

last_cell = target_sheet.range("E1").end("down")
# 選擇 走勢資料 上，B2 到 E267 這個範圍，
data = target_sheet.range(f"B2:E{last_cell.row}")
# 將圖表的資料設定成該範圍内的資料
chart.set_source_data(data)
# 最後將圖表的類別設定成 開盤-高-低-收盤股價圖
chart.chart_type = "stock_ohlc"

In [ ]:
import time

dashboard = wb.sheets["即時看板"]


# 截取即時看板上的股票代號
stock_id = dashboard.range("stock1").value

# 將被讀取成浮點數的股票代號轉成字串
if type(stock_id) == float:
    stock_id = str(int(stock_id))
        
target_sheet = wb.sheets["2884走勢資料"]

# 呼叫函數，將截取到的圖表資料（一個二維陣列）寫入以 A2 為起點的範圍
target_sheet.range("A1").value = get_chart_data(stock_id, "demo")

chart = dashboard.charts["chart1"]
last_cell = target_sheet.range("E1").end("down")
chart.set_source_data(target_sheet.range(f"B2:E{last_cell.row}"))
chart.chart_type = "stock_ohlc"
# 將最後一筆收盤價寫入看板工作表
dashboard.range("price1").value = last_cell.value
    

In [ ]:
condition = dashboard.range("condition1").value
limit = dashboard.range("limit1").value
price = dashboard.range("price1").value
stock_id = int(dashboard.range("stock1").value)

print(condition)
print(limit)
print(price)
print(stock_id)

In [ ]:
# 寫出條件判斷
def check_condition(stock_id, price, condition, limit):
    msg = ""
    if condition == "<":
        if price < limit:
             msg += "{} 的價格低於 {}\n".format(stock_id, limit)
    elif condition == ">":
        if price > limit:
             msg += "{} 的價格高於 {}\n".format(stock_id, limit)
    elif condition == "=":
        if price == limit:
             msg += "{} 的價格等於 {}\n".format(stock_id, limit)
    
    return msg

In [ ]:
condition = dashboard.range("condition1").value
limit = dashboard.range("limit1").value
price = dashboard.range("price1").value
stock_id = dashboard.range("stock1").value

check_condition(stock_id, price, condition, limit)

In [ ]:
def line_notify(msg, line_token):
    line_url = "https://notify-api.line.me/api/notify"
    line_token = "Y3PNDgDPkCCMPU6XikKEz9X4S9juY7YtDKFmciMCx4R"
  
    headers = {
            "Authorization": "Bearer " + line_token, 
            "Content-Type" : "application/x-www-form-urlencoded"
        }

    payload = {'message': msg}
    r = requests.post(line_url, headers = headers, params = payload)
    return r

In [ ]:
import time

dashboard = wb.sheets["即時看板"]

fugle_token = wb.sheets["觀察清單"].range("L1").value
line_token = wb.sheets["觀察清單"].range("L3").value

line_report = "Nielsen及時股價看板：\n\n"
# 截取即時看板上的股票代號
stock_id = dashboard.range("stock1").value

# 將被讀取成浮點數的股票代號轉成字串
if type(stock_id) == float:
    stock_id = str(int(stock_id))

target_sheet = wb.sheets["2884走勢資料"]

# 呼叫函數，將截取到的圖表資料（一個二維陣列）寫入以 A2 為起點的範圍
target_sheet.range("A2").value = get_chart_data(stock_id, fugle_token)
# 選擇即時看板的圖表物件
chart = dashboard.charts["chart1"]
last_cell = target_sheet.range("E1").end("down")
# 將圖表物件的資料來源設定成範圍 B2:GN
chart.set_source_data(target_sheet.range(f"B2:E{last_cell.row}"))
# 將圖表類型設定成 ohlc（開盤-高-低-收盤股價圖）
chart.chart_type = "stock_ohlc"
# 將最後一筆收盤價寫入看板工作表
dashboard.range("price1").value = last_cell.value
# 讀取觸發 Line 通知的比較條件
condition = dashboard.range("condition1").value
limit = dashboard.range("limit1").value
price = dashboard.range("price1").value
# 檢查是否符合觸發 Line 通知，產生 Line 訊息的字串
line_report += check_condition(stock_id, price, condition, limit)

# 發送 Line 訊息
line_notify(line_report, line_token)
